In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian
import networkx as nx
from scipy.sparse.csgraph import minimum_spanning_tree

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = adjacency_matrix.data ** 0.3
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [5]:
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
#r = np.mean(adjacency_matrix.data**2)
D, A = build_weighted_bethe_hessian(adjacency_matrix, r)

n = adjacency_matrix.shape[0]
I = scipy.sparse.eye(n, format='csr')
Hr = D - A #+ I * np.mean(degrees)

In [12]:
class MatrixOperator(object):

    def __init__(self, A):
        self.A = A.astype(PETSc.ScalarType)
        self.n_calls = 0

    def mult(self, A, x, y):
        xx = x.getArray(readonly=1)
        yy = y.getArray(readonly=0)
        yy[:] = self.A.dot(xx)
        self.n_calls += 1
        
    def getDiagonal(self, A, y):
        yy = y.getArray(readonly=0)
        yy[:] = self.A.diagonal()

In [13]:
from petsc4py import PETSc
from slepc4py import SLEPc

A = Hr
k = 10
tol = 1e-2
max_iter = 100

### Setup matrix operator
n = A.shape[0]
mat = MatrixOperator(A)
A_operator = PETSc.Mat().createPython([n, n], mat)
A_operator.setUp()

### Solve eigenproblem
E = SLEPc.EPS()
E.create()
E.setOperators(A_operator)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
E.setType(SLEPc.EPS.Type.JD)
E.setDimensions(k)
E.setTolerances(tol, max_iter)
E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

start = time.time()
E.solve()
print("Time_elapsed: %d" % (time.time() - start))
print("Number of calls to Ax: %d" % mat.n_calls)

Error: error code 76
[0] EPSSolve() line 147 in /Users/i.lobov/slepc-3.8.2/src/eps/interface/epssolve.c
[0] EPSSolve_XD() line 211 in /Users/i.lobov/slepc-3.8.2/src/eps/impls/davidson/davidson.c
[0] dvd_updateV_extrapol() line 316 in /Users/i.lobov/slepc-3.8.2/src/eps/impls/davidson/dvdupdatev.c
[0] dvd_updateV_update_gen() line 268 in /Users/i.lobov/slepc-3.8.2/src/eps/impls/davidson/dvdupdatev.c
[0] dvd_improvex_jd_gen() line 717 in /Users/i.lobov/slepc-3.8.2/src/eps/impls/davidson/dvdimprovex.c
[0] dvd_improvex_jd_proj_cuv() line 651 in /Users/i.lobov/slepc-3.8.2/src/eps/impls/davidson/dvdimprovex.c
[0] Error in external library
[0] Error in LAPACK subroutine getrf: info=1

In [9]:
### Collect results
print("")
its = E.getIterationNumber()
print("Number of iterations of the method: %i" % its)
sol_type = E.getType()
print("Solution method: %s" % sol_type)
nev, ncv, mpd = E.getDimensions()
print("NEV %d NCV %d MPD %d" % (nev, ncv, mpd))
print("Number of requested eigenvalues: %i" % nev)
tol, maxit = E.getTolerances()
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
nconv = E.getConverged()
print("Number of converged eigenpairs: %d" % nconv)
nconv = min(nconv, k)

vecs = np.zeros([n, nconv])
vals = np.zeros(nconv)

xr, tmp = A_operator.getVecs()
xi, tmp = A_operator.getVecs()

if nconv > 0:
    for i in range(nconv):
        k = E.getEigenpair(i, xr, xi)
        vals[i] = k.real
        vecs[:, i] = xr


Number of iterations of the method: 16
Solution method: krylovschur
NEV 100 NCV 200 MPD 200
Number of requested eigenvalues: 100
Stopping condition: tol=0.01, maxit=100
Number of converged eigenpairs: 107


In [21]:
D_inv = D.copy()
D_inv.data = 1.0 / (D_inv.data + np.mean(degrees))

In [22]:
rng = np.random.RandomState(0)
dim = 100
init = rng.rand(n, dim)

start = time.time()
vals, vecs = lobpcg(Hr, M=D_inv, X=init, maxiter=100, largest=False, verbosityLevel=1)
print("Time elapsed: %d" % (time.time() - start))

Solving generalized eigenvalue problem with preconditioning

matrix size 189533
block size 100

No constraints


iteration 0
current block size: 100
eigenvalue: [  715.42614982  1249.84041508  1251.66617848  1252.90308503  1256.11959769
  1257.8041908   1258.36402826  1259.50419787  1261.0945551   1262.12944396
  1262.68965938  1263.69618549  1264.65596457  1265.90428594  1267.33722813
  1268.38488861  1270.13773664  1270.59759776  1271.40763256  1271.87455177
  1272.52114597  1274.23376883  1275.35600307  1276.15345987  1277.01534931
  1277.86359226  1278.71647702  1280.81120541  1281.10353039  1281.85296452
  1283.02794408  1283.24829008  1284.61455909  1285.09182558  1286.08141472
  1288.52328008  1289.51962514  1289.91587748  1291.21536809  1291.78875052
  1293.42624925  1294.83041903  1295.9215139   1296.35630271  1297.93806793
  1298.17159451  1299.89922827  1300.88972628  1302.46530106  1303.957402
  1304.53575379  1305.75478607  1305.77532132  1308.77264638  1309.76261915
  131

iteration 3
current block size: 100
eigenvalue: [ 714.12340988  910.64876442  916.24224041  931.31292806  934.15082312
  935.70524831  936.39520891  938.67188617  940.84746073  942.08760787
  943.43099147  945.17152848  946.03760188  947.0720174   947.43579422
  948.27038497  949.62162187  949.72036319  950.41170981  951.10971362
  951.13613703  951.39871404  951.87924303  952.02667415  952.17331996
  952.37411125  952.56440665  952.74001873  952.93051165  953.40740237
  953.49384126  953.67745254  953.76943876  953.97056137  954.05597381
  954.27433984  954.42337234  954.49095085  954.55489398  954.6604436
  954.76227067  954.78479523  954.87764126  954.88142574  955.16919441
  955.26196167  955.35378166  955.43425289  955.50121622  955.57086498
  955.59616669  955.66395371  955.79971589  955.82061746  955.85689929
  955.89554449  955.95786258  955.98442386  955.9950556   956.01305659
  956.04547733  956.05774603  956.10462213  956.12105745  956.1304397
  956.1454022   956.17378816  9

iteration 6
current block size: 100
eigenvalue: [ 713.78673481  909.96454304  914.07079788  927.17814709  928.38395121
  929.95949887  930.48237515  933.17394741  936.20697266  936.88423093
  937.4167008   938.54868818  939.96341814  941.19097926  942.14175683
  942.43408483  943.35530326  944.44738393  945.30574187  945.98141763
  946.17326501  946.97235723  947.31943414  947.46061397  947.96419733
  948.57879208  949.23058051  949.29404445  949.73714662  950.03970112
  950.20738232  950.47344155  950.93882072  950.96750476  951.45015613
  951.60239228  951.78711581  951.8364273   952.08556431  952.1188101
  952.21725205  952.62354498  952.77024516  952.82283631  952.86871543
  953.12591706  953.33138837  953.48632867  953.60321458  953.72349381
  953.76671227  953.98105206  954.08371005  954.14752521  954.27472455
  954.39756823  954.48232242  954.5983832   954.66993785  954.73959665
  954.78157027  954.85476565  954.99051635  955.12573057  955.20275618
  955.24827291  955.30971968  

iteration 9
current block size: 100
eigenvalue: [ 713.63198484  909.79885337  913.59289936  925.42163822  927.07201992
  928.12088138  928.5044432   931.97408133  934.54277529  934.96645048
  936.03160149  937.05960257  937.87762021  939.79682478  939.89482195
  940.69304399  940.98898217  942.74792798  943.26586513  944.2182029
  944.8414785   945.44156415  945.63174206  945.91490024  946.65200536
  947.51355642  947.59220526  947.97615945  948.31854812  948.67151373
  949.14326313  949.46619539  949.57338585  949.69394957  950.3047923
  950.7197378   951.05798871  951.14052058  951.24623983  951.50611619
  951.74124248  951.82996874  951.85025922  952.01888816  952.26940379
  952.40231774  952.70791388  952.85174831  953.13582569  953.21150094
  953.43764353  953.4817179   953.57930817  953.69753468  953.84305458
  953.87590802  954.03186639  954.08052491  954.19976074  954.28165593
  954.32632991  954.4147298   954.57134336  954.61595078  954.71827347
  954.79782945  954.9349167   9

KeyboardInterrupt: 

In [ ]:
### LOBPCG for 10 eigenvalues
### Default - 143 seconds
### Jacobi - 59 seconds
### Spanning tree with inversed values - does not converge...

In [ ]:
output_path = "../data/wiki/win=1_correctly_weighted_bethe_hessian_lobpcg_pow=0.3_dim=100"
np.save(output_path + ".vecs", vecs[:,:100])
np.save(output_path + ".vals", vals[:100])
np.save(output_path + ".degrees", degrees)

In [10]:
import shutil
base_path = '../data/wiki/'
shutil.copyfile(base_path + 'win=1_weighted_bethe_hessian_slepc_scaled_abs_tol=1e-3_pow=0.3_dim=500.words.vocab', 
                base_path + 'win=1_weighted_bethe_hessian_slepc_middle_scaling_pow=0.3_dim=100' + ".words.vocab")

'../data/wiki/win=1_weighted_bethe_hessian_slepc_middle_scaling_pow=0.3_dim=100.words.vocab'